<a href="https://colab.research.google.com/github/eriramirez/ProgFin/blob/main/004.1_session%20international%20portfolio%20optimization%202024a.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Optimization of an International Portfolio

In [2]:
! pip install cvxpy
! pip install riskfolio-lib


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.6/307.6 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 34.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 14.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.3/13.3 MB 77.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 983.4/983.4 kB 54.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 159.9/159.9 kB 11.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.0/235.0 kB 18.1 MB/s eta 0:00:00
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.2.2
    Uninstalling scikit-learn-1.2.2:
      Successfully uninstalled scikit-learn-1.2.2
  Attempting uninstall: cvxpy
    Found existing installation: cvxpy 1.3.4
    Uninstalling cvxpy-1.3.4:
      Successfully uninstalled cvxpy-1.3.4


In [3]:
# Download .csv file to colab instance
! wget https://raw.githubusercontent.com/eriramirez/ProgFin/main/ACTIVOS%20Actinver%20PAMA%20para%20python.csv

--2024-06-04 01:18:30--  https://raw.githubusercontent.com/eriramirez/ProgFin/main/ACTIVOS%20Actinver%20PAMA%20para%20python.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.111.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 18347374 (17M) [text/plain]
Saving to: ‘ACTIVOS Actinver PAMA para python.csv’

ACTIVOS Actinver PA 100%[===================>]  17.50M  41.8MB/s    in 0.4s    

2024-06-04 01:18:32 (41.8 MB/s) - ‘ACTIVOS Actinver PAMA para python.csv’ saved [18347374/18347374]



In [4]:
# Import libraries

import pandas as pd
from datetime import datetime, timedelta
import numpy as np
from typing import Dict, List
import cvxpy as cp
import riskfolio as rp


In [6]:
df = pd.read_csv ("ACTIVOS Actinver PAMA para python.csv")
# df["Date"] = pd.to_datetime(df["Date"], format='mixed', dayfirst=True).to_pydatetime()
df["Date"] = df["Date"].apply(lambda x: datetime.strptime(x, "%d/%m/%Y"))
df.index=df.Date
df

NameError: name 'datetime' is not defined

In [ ]:
# drop columns with no information
df = df.dropna(axis=1,how='all')
df

In [ ]:
# Drop old dates to analyze in Riskfolio
from datetime import datetime, timedelta
start = datetime(2019,1,1)
df = df[df["Date"]>start]
df

In [ ]:
# Import the necessary libraries
from datetime import datetime, timedelta
import pandas as pd
import numpy as np
from typing import Dict, List
import cvxpy as cp
import riskfolio as rp
import warnings

In [ ]:
assets = ["AAL","AAPL","AAU", "WMT"]
Y = df[assets].pct_change().dropna()
display(Y.head())
#pct_change_df = data.pct_change()

In [ ]:
# Building the portfolio object
port = rp.Portfolio(returns = Y)

#Calculating optimal portfolio

#Select method and estimate input parameters
method_mu = 'hist' # method to estimate expected returns based on historical data
method_cov = 'hist' # method to estimate covariance matrix based on historical data

port.assets_stats(method_mu = method_mu, method_cov = method_cov, d = 0.94)

#Estimate optimal portfolio
model = "Classic" # Could be Classic (historical), BL (Black Litterman), or FM (FActor Model)
rm = "MV" # Risk measure used, this time will be variance
obj = 'Sharpe' # Objective function, could be MinRisk, MaxRet, Utility of Sharpe
hist = True # Use historical scenarios for risk measures that depend on scenarios
rf = 0 # Risk Free Rate
l = 0 # Risk aversion factor, unly useful when objective is 'Utility'

w = port.optimization(model=model, obj=obj, rf=rf, l=l, hist=hist)

display(w.T)

In [ ]:
rp.Reports.jupyter_report(Y, w)

In [ ]:
rp.Reports.excel_report(
    Y,
    w,
    rf=0,
    alpha=0.05,
    t_factor=252,
    ini_days=1,
    days_per_year=252,
    name=r"C:\Users\L00616607\Developer\ProgFin\python port1.xlsx",
)


In [ ]:
# Ploting the composition of the portfolio
ax = rp.plot_pie(w=w, title='Sharpe Mean Variance', others=0.05, nrow=len(data.columns),
                cmap='tab20', height=6, width=10, ax=None)

In [ ]:
ann_mean_return = ((1 + Y.mean())**252)-1
print(ann_mean_return)

In [ ]:
ann_stdev = Y.std() * (252**(1/2))
print(ann_stdev)

In [ ]:
Y.cov()

In [ ]:
w.round(4)

In [ ]:

#weight = np.full_like(ann_stdev,1/len(ann_stdev))
#weight

In [ ]:
portaf_ann_stdev = ((w.T @ Y.cov()) @ w) ** (1/2)
portaf_ann_stdev

In [ ]:
df = pd.DataFrame({"goog":goog.Close, "cemex":cemex.Close, 'mxn':mxn['Close']})
# drop rows where all columns in subset are NA
df = df.dropna(axis=0, how='all', subset=('goog', 'cemex'))
# fill with the value of the previous date's values
df = df.fillna(method='ffill')
# in case there are empty rows at the beginning, fill with following date's values
df = df.fillna(method='bfill')

In [ ]:
#Add a column with google prices in MXN
df['goog_mxn'] = df.goog * df.mxn

In [ ]:
df.plot(y=['goog_mxn', "cemex"])

In [ ]:
df.corr()

In [ ]:
df.cov()

In [ ]:
df.std()

In [ ]:
df.std() * np.sqrt(252)

In [ ]:
df['goog_mxn_change'] = df.goog_mxn.pct_change()

In [ ]:
df.to_excel('example.xlsx')